In [1]:
import duckdb
import pandas as pd
import csv

In [2]:
# create a connection to the database
con = duckdb.connect('/workspaces/practice/terminology.db')

In [3]:
con.execute('create schema if not exists medispan')

In [4]:
con.execute('''
    create or replace table medispan.fgpfile(
        external_drug_identifier         VARCHAR(30)   NOT NULL,
        price_type                       VARCHAR(2)    NOT NULL,
        pharmacy_type                    VARCHAR(5)    NOT NULL,
        price_effective_date             INT            NOT NULL,
        transaction_cd                   VARCHAR(1),
        unit_price                       VARCHAR(21)   NOT NULL,
        reserve_1                        VARCHAR(2),
        reserve_2                        VARCHAR(2),
        nadac_explanation_code           VARCHAR(20)
    )
''')

con.execute('''
    copy medispan.fgpfile from '/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/FGPFILE'(
        delimiter '|',
        header 'false'
    )
''')

# show first 5 rows of table
con.execute('select * from medispan.fgpfile limit 5').fetchdf()

,external_drug_identifier,price_type,pharmacy_type,price_effective_date,transaction_cd,unit_price,reserve_1,reserve_2,nadac_explanation_code
0,00002322730,7,Z,20210201,NaN,0.699198,NaN,NaN,NaN
1,00002322730,7,Z,20210301,NaN,0.715743,NaN,NaN,NaN
2,00002322730,7,Z,20210401,NaN,0.670885,NaN,NaN,NaN
3,00002322730,7,Z,20210501,NaN,0.696969,NaN,NaN,NaN
4,00002322730,7,Z,20210601,NaN,0.722461,NaN,NaN,NaN


In [5]:
cols = ['price_type', 'aberrant_explanation_code', 'transaction_cd', 'aberrant_explanation_code_desc']

with open('/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/fgpabex', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    data = list(reader)
    fgpabex = pd.DataFrame(data, columns=cols)

con.execute('create or replace table medispan.fgpfile as select * from fgpabex')

con.execute('select * from medispan.fgpabex limit 5').fetchdf()

,price_type,aberrant_explanation_code,transaction_cd,aberrant_explanation_code_desc


In [6]:
cols = ['external_drug_identifier', 'pharmacy_type', 'transaction_cd', 'description', 'otc_flag', 'pricing_unit', 'class_rate_setting', 'in_medispan_db_flag', 'medispan_ndc_inactive_dt', 'is_same_mddb_pkg_sum']

with open('/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/FGPNADD', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    data = list(reader)
    fgpnadd = pd.DataFrame(data, columns=cols)

con.execute('create or replace table medispan.fgpnadd as select * from fgpnadd')

con.execute('select * from medispan.fgpnadd limit 5').fetchdf()

,external_drug_identifier,pharmacy_type,transaction_cd,description,otc_flag,pricing_unit,class_rate_setting,in_medispan_db_flag,medispan_ndc_inactive_dt,is_same_mddb_pkg_sum
0,00002143380,C,,TRULICITY 0.75 MG/0.5 ML PEN,N,ML,B,Y,,Y
1,00002143380,I,,TRULICITY 0.75 MG/0.5 ML PEN,N,ML,B,Y,,Y
2,00002143480,C,,TRULICITY 1.5 MG/0.5 ML PEN,N,ML,B,Y,,Y
3,00002143480,I,,TRULICITY 1.5 MG/0.5 ML PEN,N,ML,B,Y,,Y
4,00002143601,C,,EMGALITY 120 MG/ML PEN,N,ML,B,Y,,Y


In [7]:
cols = ['external_drug_identifier', 'transaction_code', 'product_group', 'ingredient', 'strength', 'dosage', 'route', 'mdr_unit_type', 'package_size', 'reserve_1', 'reserve_2', 'reserve_3', 'a_rated', 'reserve_4', 'reserve_5', 'in_medispan_db_flag', 'medispan_ndc_inactive_dt', 'is_same_mddb_pkg_sum', 'calculation_basis']

with open('/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/fgpfuld', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    data = list(reader)
    fgpfuld = pd.DataFrame(data, columns=cols)

con.execute('create or replace table medispan.fgpfuld as select * from fgpfuld')

con.execute('select * from medispan.fgpfuld limit 5').fetchdf()

,external_drug_identifier,transaction_code,product_group,ingredient,strength,dosage,route,mdr_unit_type,package_size,reserve_1,reserve_2,reserve_3,a_rated,reserve_4,reserve_5,in_medispan_db_flag,medispan_ndc_inactive_dt,is_same_mddb_pkg_sum,calculation_basis
0,00002322730,,3057,ATOMOXETINE HYDROCHLORIDE,10MG,CAPSULE,ORAL,CAP,30.0,,,,Yes,,,Y,,N,Y
1,00002322830,,3060,ATOMOXETINE HYDROCHLORIDE,25MG,CAPSULE,ORAL,CAP,30.0,,,,Yes,,,Y,,N,N
2,00002322930,,3061,ATOMOXETINE HYDROCHLORIDE,40MG,CAPSULE,ORAL,CAP,30.0,,,,Yes,,,Y,,N,N
3,00002323030,,7975,OLANZAPINE/FLUOXETINE HYDROCHLORIDE,3MG-25MG,CAPSULE,ORAL,CAP,30.0,,,,Yes,,,Y,,N,Y
4,00002323130,,4792,OLANZAPINE/FLUOXETINE HYDROCHLORIDE,6MG-25MG,CAPSULE,ORAL,CAP,30.0,,,,Yes,,,Y,,N,Y


In [8]:
cols = ['record_type', 'reserve_1', 'sequence_number', 'reserve_2', 'comment_marker', 'data_comment']

fgpsum = pd.read_csv('/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/fgpsum', sep='|', names=cols)

con.execute('create or replace table medispan.fgpsum as select * from fgpsum')

con.execute('select * from medispan.fgpsum limit 5').fetchdf()

,record_type,reserve_1,sequence_number,reserve_2,comment_marker,data_comment
0,CCR,NaN,10,NaN,NaN,"Copyright 2021 UpToDate, Inc. All Rights Rese..."
1,CDC,NaN,10,NaN,*,Create Date
2,CDC,NaN,20,NaN,NaN,20211125
3,CDE,NaN,10,NaN,*,Expiration Date
4,CDE,NaN,20,NaN,NaN,20211208


In [9]:
cols = ['field_identifier', 'field_description', 'field_type', 'field_length', 'implied_decimal_flag', 'decimal_places', 'field_validation_flag', 'field_abbreviation_flag', 'reserve']

with open('/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/fgpdict', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    data = list(reader)
    fgpdict = pd.DataFrame(data, columns=cols)

con.execute('create or replace table medispan.fgpdict as select * from fgpdict')

con.execute('select * from medispan.fgpdict limit 5').fetchdf()

,field_identifier,field_description,field_type,field_length,implied_decimal_flag,decimal_places,field_validation_flag,field_abbreviation_flag,reserve
0,A001,Record Type,C,003,N,00,N,N,
1,A004,Reserve-1,C,001,N,00,N,N,
2,A005,Sequence Number,N,003,N,00,N,N,
3,A008,Reserve-2,C,001,N,00,N,N,
4,A009,Comment Marker,C,001,N,00,N,N,


In [10]:
cols = ['field_identifier', 'field_value', 'language_code', 'value_description', 'value_abbreviation', 'reserve']

with open('/workspaces/practice/terminology/medispan/fedgovprc_0_0_wk_pdu-delimt_1.x_d_20211201/FGP/USAENG/DB/fgpval', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    data = list(reader)
    fgpval = pd.DataFrame(data, columns=cols)

con.execute('create or replace table medispan.fgpval as select * from fgpval')

con.execute('select * from medispan.fgpval limit 5').fetchdf()

,field_identifier,field_value,language_code,value_description,value_abbreviation,reserve
0,C040,C,01,Character,,
1,C040,N,01,Number,,
2,C044,N,01,No,,
3,C044,Y,01,Yes,,
4,C047,N,01,No,,


In [11]:
con.close()